# Two-Phase Flash Calculation (VLE and LLE)

The isothermal-isobaric two-phase flash is carried out by a combination of accelerated successive substitution (ASS) to update phase compositions and Halley's method to solve the Rachford-Rice mass balance. 

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1+\psi (K_i-1)} $$

Where,  $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta  $ represents the equilibrium constant  and $\psi$ the fraction of the phase $\beta$. The described method can be slow at high pressures, for that reason, the number of cycles of ASS is limited to ``nacc`` cycles and if no solution is reached the algorithm changes to a second-order procedure based on Gibbs free energy minimization:

$$ min \, {G(\underline{F}^\alpha, \underline{F}^\beta)} = \sum_{i=1}^c (F_i^\alpha \ln \hat{f}_i^\alpha + F_i^\beta \ln \hat{f}_i^\beta) $$

Here, $F$ refers to the number of moles and $\hat{f}$ to the effective fugacity, the superscript refers to the phase index and the subscript to the specie index. The optimization is performed using SciPy minimization routines.

SGTPy ``flash`` routine solves phase compositions for two-phase flash at constant pressure and temperature (PT-flash).  The ```flash``` function requires initial guesses for the phase compositions, the states of the phases, i.e ```LV``` fo liquid-vapor flash or ```LL``` for liquid-liquid flash, the global phase composition (``z``),  the temperature (``T``) and pressure (``P``) of the system.

In this notebook, Vapor-Liquid and Liquid-Liquid flash calculation will be exemplified. To start, the required functions are imported.

In [1]:
import numpy as np
from sgtpy import component, mixture, saftvrmie
from sgtpy.equilibrium import flash

---
### Vapor Liquid Equilibrium Flash

This calculation will be exemplified for the mixture of ethanol and water.

In [2]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6.,  eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515455e-20)

ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

mix = mixture(ethanol, water)
# or 
mix = ethanol + water

kij, lij = np.array([-0.0069751 , -0.01521566])

Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])
# setting interactions corrections
mix.kij_saft(Kij)
mix.lij_saft(Lij)

# or setting interaction between component i=0 (ethanol) and j=1 (water)
mix.set_kijsaft(i=0, j=1, kij0=kij)
mix.set_lijsaft(i=0, j=1, lij0=lij)

# creating eos model
eos_vle = saftvrmie(mix)
# flash conditions
T = 355. # K
P = 1e5 # Pa
z = np.array([0.4, 0.6]) # global composition
# initial guesses
x0 = np.array([0.2, 0.8]) 
y0 = np.array([0.6, 0.4])

#liquid phase composition, vapor phase composition and vapor phase fraction.
flash(x0, y0, 'LV', z, T, P, eos_vle)

(array([0.26353599, 0.73646401]),
 array([0.55807907, 0.44192093]),
 0.4633074753995569)

Optionally, you can supply initial guesses for the phase volumes (``v0``) or the non-bonded association sites solution (``Xass0``). These initial guesses can be obtained from previous computations and the ``full_output=True`` option.

In [3]:
# flash conditions
T = 355. # K
P = 1e5 # Pa
z = np.array([0.4, 0.6]) # global composition
# initial guesses
x0 = np.array([0.2, 0.8]) 
y0 = np.array([0.6, 0.4])
sol_vle = flash(x0, y0, 'LV', z, T, P, eos_vle, full_output=True)
sol_vle

      T: 355.0
      P: 100000.0
   beta: 0.4633074753995569
  error: 6.866923015729792e-09
   iter: 13
      X: array([0.26353599, 0.73646401])
     v1: 2.9414097098116e-05
  Xass1: array([0.03669632, 0.10345249, 0.10315897, 0.19817013])
 state1: 'L'
      Y: array([0.55807907, 0.44192093])
     v2: 0.028136298011136793
  Xass2: array([0.9410755 , 0.96994287, 0.97517731, 0.98397015])
 state2: 'V'

In [4]:
# flash conditions
T = 355. # K
P = 1e5 # Pa
z = np.array([0.4, 0.6]) # global composition
# initial guesses
x0 = np.array([0.2, 0.8]) 
y0 = np.array([0.6, 0.4])
v0 = [sol_vle.v1, sol_vle.v2]
Xass0 = [sol_vle.Xass1, sol_vle.Xass2]
# Flash supplying initial guess for volumes and non-bonded association sites fractions
flash(x0, y0, 'LV', z, T, P, eos_vle, v0=v0, Xass0=Xass0, full_output=True)

      T: 355.0
      P: 100000.0
   beta: 0.4633074753995016
  error: 6.866923012945472e-09
   iter: 13
      X: array([0.26353599, 0.73646401])
     v1: 2.9414097098117087e-05
  Xass1: array([0.03669632, 0.10345249, 0.10315897, 0.19817013])
 state1: 'L'
      Y: array([0.55807907, 0.44192093])
     v2: 0.02813629801113682
  Xass2: array([0.9410755 , 0.96994287, 0.97517731, 0.98397015])
 state2: 'V'

---
### Liquid Liquid Equilibrium Flash

This calculation will be exemplified for the mixture of water and butanol.

In [5]:
# creating pure components
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)
mix = mixture(water, butanol)
# or
mix = water + butanol

# optimized from experimental LLE
kij, lij = np.array([-0.00736075, -0.00737153])
Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])

# or setting interaction between component i=0 (water) and j=1 (butanol)
mix.set_kijsaft(i=0, j=1, kij0=kij)
mix.set_lijsaft(i=0, j=1, lij0=lij)

# setting interactions corrections
mix.kij_saft(Kij)
mix.lij_saft(Lij)
# creating eos model
eos = saftvrmie(mix)

T = 298.15 # K
P = 1.01325e5 # Pa

# global composition
z = np.array([0.8,0.2])
# initial guesses for the liquid phase composition
x0 = np.array([0.9, 0.1])
w0 = np.array([0.6, 0.4])

#liquid 1 phase composition, liquid phase 2 composition and liquid 2 phase fraction.
flash(x0, w0, 'LL', z, T, P, eos, full_output=True)

      T: 298.15
      P: 101325.0
   beta: 0.3757301459568324
  error: 7.032367481252242e-09
   iter: 16
      X: array([0.96021656, 0.03978344])
     v1: 2.031457633354282e-05
  Xass1: array([0.05681864, 0.07391832, 0.01687673, 0.03815106])
 state1: 'L'
      Y: array([0.53380265, 0.46619735])
     v2: 5.135704698290874e-05
  Xass2: array([0.06370272, 0.19811158, 0.02977199, 0.1168081 ])
 state2: 'L'

---
For further information about the flash function check out the documentation running: ``flash?``